<a href="https://colab.research.google.com/github/jcandane/StochasticPhysics/blob/main/autoexpFVGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy
import gpcam
from gpcam.autonomous_experimenter import AutonomousExperimenterFvGP

import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import plotly.graph_objects as go

def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y,z=z))
    if data is not None:
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers'))

    fig.update_layout(title='Posterior Mean', autosize=True,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90))


    fig.show()
    return None

https://gpcam.readthedocs.io/en/latest/examples/advanced_test.html#running-a-multi-task-gp-autonomous-data-acquisition


In [ ]:
data = np.load("sarcos.npy") ###np.load("./data/sarcos.npy")
print(data.shape)
x = data[:,0:21]
y = data[:,21:23]

(4449, 28)


## here we run fvGP with RCF!!

In [ ]:
import h5py ## https://docs.h5py.org/en/stable/quick.html


#### inputspace
input_s = np.array([np.array([np.min(x[:,i]),np.max(x[:,i])]) for i in range(len(x[0]))])
print("index set (input space) bounds:")
print(input_s)

domain = np.array([[ 0.0, 1.0],
                   [-0.5,  0.7]])
input_s = domain

from RCF import RCF ### !!!!!

dr_x   = np.array([0.1])
domain = np.array([[ 0.0, 10.0]])

f = RCF(input_s, 2, D=2, ξ=2)


def instrument(data, instrument_dict = {}):
    for entry in data:
        #print("Suggested by gpCAM: ", entry["x_data"])
        #entry["y_data"] = griddata(x,y,entry["x_data"],method = "nearest", fill_value = 0)[0]
        entry["y_data"] = f.evalulate(np.asarray([entry["x_data"]])) #.flatten() ##
        entry["output positions"] = np.array([[0],[1]])
        #print("received: ", entry["y_data"])
    print("")
    return data


#################################
def math_function(x_data):
    return f.evalulate(x_data)

def gpcam_to_h5(data, filename="to_vintrument.h5"):
    """ this function reads gpcam's data, and creates an h5 file (to be read by the instrument)
    GIVEN   > data : List[dict] (gpCAM dataset datatype, !contains various datatypes)
              **filename : str (optional, str specifying the output h5 file)
    GET     > None
    """

    to_analyze=[]
    for entry in data:
        to_analyze.append(entry["x_data"])
    to_analyze = np.asarray(to_analyze) ## make into a np.array, D_ax

    h5f = h5py.File(filename, "w")
    h5f.create_dataset("dataset_1", data=to_analyze)
    h5f.close()
    return None

def h5_to_vinstrument(filename="to_vintrument.h5"):
    """
    this function reads a h5 file, to obtain a 2d-numpy.array (to be used by the virtual-intrument)
    GIVEN > **filename : str
    GET   > x_data : np.ndarray{2d} (D_ax, 1st-index enumerates snapshots, 2nd-index enumerates IN-coordiante, i.e. D_ax)
    """

    h5f    = h5py.File(filename, "r")
    x_data = np.asarray(h5f.get('dataset_1'))
    h5f.close()
    return x_data ### numpy.array of dimensions ( samples , coordinates ) i.e. D_ax

def vinstrument_to_h5(y_data, filename="from_vintrument.h5"):
    """
    this function obtained the vintrument's y_data, along with other meta-data saves to an h5
    GIVEN > y_data : np.ndarray{2d} (2d-np.array, 1st-index : data-entry number, 2nd-index : OUT-coordinate, i.e. D_aX)
    GET >   None
    """

    h5f = h5py.File(filename, "w")
    h5f.create_dataset("dataset_1", data=y_data)
    h5f.close()
    return None

def h5_to_gpcam(data, filename="from_vintrument.h5"):
    """ this function updates gpcam's "data" variable (List[dict]), by reading a h5 file.
    GIVEN > data : List[dict] (gpCAM dataset datatype, !contains various datatypes)
            **filename : str (optional, str specifying the input h5 file)
    GET   > data : List[dict] (gpCAM dataset datatype, !contains various datatypes)
    """
    h5f    = h5py.File(filename, "r")
    y_data = np.asarray(h5f["dataset_1"]) ## D_aX
    h5f.close()

    for a, entry in enumerate(data):
        entry["y_data"] = np.asarray([y_data[a]]) ### this should have the shape of (2,1) as given in instrument
        entry["output positions"] = np.asarray([np.arange(len(y_data[a]))]).T #np.array([[0],[1]]) ### this is important for fvGP object!

    return data

def vinstrument():
    """ python-function for virtual-instrument, reads and writes h5 files
    GIVEN > None
    GET   > None
    """

    x_data = h5_to_vinstrument()

    y_data = math_function(x_data)

    vinstrument_to_h5(y_data)

    return None

def Instrument(data):

    ### gpcam -> h5 (x-coordinates only)
    gpcam_to_h5(data)

    ### vintrument()
    vinstrument()

    ### h5 -> gpcam (everything)
    data = h5_to_gpcam(data)

    return data
#################################

my_fvae = AutonomousExperimenterFvGP(input_s,2,1,
                                     init_dataset_size= 25, instrument_function = instrument
                                     ) ##acquisition_function=acq_func
print("-----------------------")

my_fvaee = AutonomousExperimenterFvGP(input_s,2,1,
                                     init_dataset_size= 25, instrument_function = Instrument
                                     ) ##acquisition_function=acq_func

print("-----------------------")
my_fvaee.train()
print("-----------------------")
my_fvaee.go(50) #, retrain_async_at=(22,), retrain_globally_at=(50,90,120), retrain_locally_at=(25,))

index set (input space) bounds:
[[ -0.842481   0.661032]
 [ -0.939933  -0.085018]
 [ -0.46773    0.359348]
 [  0.797788   2.239407]
 [ -0.242241   1.278097]
 [ -0.919895   0.369078]
 [ -0.296364   0.686928]
 [ -4.771399   4.488624]
 [ -1.634053   2.289369]
 [ -2.884804   2.558282]
 [ -4.196409   3.734968]
 [ -3.067483   2.380553]
 [ -2.433971   1.978575]
 [ -3.180643   2.568279]
 [-48.072386  48.872604]
 [-25.585054  25.225171]
 [-24.697862  26.106756]
 [-36.19139   71.176937]
 [-38.485967  35.804308]
 [-22.103174  17.84188 ]
 [-36.502723  30.806254]]

-----------------------
-----------------------
-----------------------
